# Masterthesis
## Hyperparameter Optimization

This script find out the hyperparameter for the each machine learning algorithmus and scenario.  
HalvingGridSearch was used to determine the hyperparameters. It is also possible to determine the hyperparameters for individual machine learning algorithms and a single scenario.

**Imports and Definitions**
- The necessary libraries are loaded here and important variables are defined

**Imports and settings for this script**
- Import libraries and set variables for this script

**Preparation for Hyperparameter Search with HalvingGridSearchCV**
- Define the machine learning algorithm, set parameter and create pipline for the HalvingGridSearch

**HalvingGridSearch**
- Read Scenario config files and run all Scenarios with the estimatiors to derterminate Hyperparameter

**HalvingGridSearch for a single file**
- Used to Search Hyperparameter for a single Scenario file

## Imports and Definitions

In [ ]:
# Import sklearn
import sklearn

# Import pandas
import pandas as pd

# Import numpy
import numpy as np

# To calculate amplitude and phase
import math

# Measure runtime of a jupyter jotebook code cell
from timeit import default_timer as timer

# Used to check if file exists
import os

# Used to check if directory exists
import pathlib

# Import Operation System Calls
import SubOperationSystem

# check os
if os.name == 'nt':
    print("OS is Windows")
    Delimiter = '\\'
    
else:
    print("OS is Linux")
    Delimiter = '/'
    
# Path of datasets (root directory)
PathDataset = 'Dataset' + Delimiter    

# Path of datasets
PathDatasetSub = PathDataset + 'CsiFilesRah' + Delimiter
        
# Path of the converted files
PathConverted = PathDataset + 'Converted' + Delimiter

# Set path for scenario files
PathScenario = PathDataset + 'Scenario' + Delimiter

# Set path for scenario files
PathResult = PathDataset + 'Result' + Delimiter

# Set path for scenario files
PathPlot = PathDataset + 'Plot' + Delimiter

# Set path for scenario files
PathConfig = 'FilesConfig' + Delimiter

# Scenariofile (file with info about the ten scenarios)
FileScenario = 'FileScenario.csv'

# Mappingfile (file with info about original and converted filenames)
FileMapping = 'FileMapping.csv'

# Imports and settings for this script

In [ ]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Import Classifier RF and GBC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

# Import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

# Import GradientBoostingClassifier 
from sklearn.ensemble import GradientBoostingClassifier

# Import Support Vektor Machine
from sklearn import svm

# Import HalvingGridSearch
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV

# Import make_classification
from sklearn.datasets import make_classification

# Import Pipeline
from sklearn.pipeline import Pipeline

# Import AccuraceScore
from sklearn.metrics import accuracy_score

# Import PCA for Dimenstion Reduction
from sklearn.decomposition import PCA

# Import decomposition
from sklearn import decomposition

# Import standardscaler
from sklearn.preprocessing import StandardScaler

# Import f1_score
from sklearn.metrics import f1_score

# Import explainable boosting machine
from interpret.glassbox import ExplainableBoostingClassifier

# Import XGBoosting classifier
import xgboost as xgb
from xgboost import XGBClassifier

# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Preparation for Hyperparameter Search with HalvingGridSearchCV

In [ ]:
# Counter for estiminator name
EstimatorNamesCounter = 0

# Set level for verbose
vb = 0

# Set Count of crossvalidation
cv = 3

# Set random state
rs = 10

# Set Type of scoring
scoring='accuracy'

# Range of parameter
ParameterRange = [1, 2, 3, 4, 5, 6]

# Range of Parameter only for Logistic Regression
ParameterRangeLR = [1.0, 0.5, 0.1]

# Set Count for estimators
ParametersEstimators = [50,100,150,200,250]

# Set Counts of interations
ParameterInterations = 2500

# PCA parameter
ParameterPca = 30

# Set Used cores for estimators
jobs = 24

# EBM
ParameterEbmMaxBins =  [256,512,1024]
ParameterEbmMaxInteractionBins =  [16, 32, 64]
ParameterEbmMaxRounds =  [5000]
ParameterEbmLearningRate = [0.02, 0.01, 0.005]
ParameterEbmMinSampleLeaf = [2, 3]
ParameterEbmOuterBags = [14, 25, 50, 75, 100]
ParameterEbmValidationSize = [0.1, 0.15, 0.2]

# XGB
ParameterXgbeta = [0.01, 0.1, 0.3, 0.5]
ParameterXgbmax_depth = [1, 3, 5, 7, 9]
ParameterXgbmin_child_weight = [1, 3, 5, 7]
ParameterXgbsubsample = [0.2, 0.5, 0.7, 1.0]
ParameterXgbcolsample_bytree = [0.5, 0.7, 1.0]
ParameterXgbgamma = [0, 0.1, 0.3, 0.5]
ParameterXgbn_estimators = [10, 50, 100, 200]

# Define estimator names
EstimatorNames = {
    0: 'RandomForestClassifier', 
    1: 'LogisticRegression', 
    2: 'KNeighborsClassifier', 
    3: 'SVC', 
    4: 'GradientBoostingClassifier',
    5: 'ExplainableBoostingClassifier',
    6: 'XGBClassifier'
}

#####################################################################################################

# set grip parameter for RandomForestClassifier
ParamRFC = [
    {
        'RF__min_samples_leaf': ParameterRange,
        'RF__min_samples_split': ParameterRange[1:],
        'RF__max_depth': ParameterRange,
        'RF__n_estimators': ParametersEstimators
    }
]

# set grip parameter for Logistic Regression
ParamLR = [
    {
        'LR__C': ParameterRangeLR,
        'LR__solver': ['liblinear'],
        'LR__penalty': ['l1', 'l2'],
        'LR__max_iter': [ParameterInterations]
    }
]

# set grip parameter for K-NeighborsClassifier
ParamKNN = [
    {
        'KNN__weights': ['uniform', 'distance'],
        'KNN__metric': ['euclidean', 'manhattan'],
        'KNN__n_neighbors': ParameterRange,
    }
]

# set grip parameter for Support Vektor Machine
ParamSVC = [
    {
        'SVM__C': ParameterRange,
        'SVM__gamma': ['auto'],
        'SVM__kernel': ['linear', 'rbf'], 
    }
]

# set grip parameter for GradientBoostingClassifier
ParamGBC = [
    {
        'GBC__min_samples_leaf': ParameterRange,
        'GBC__max_depth': ParameterRange,
        'GBC__min_samples_split': ParameterRange[1:]
    }
]

# set grip parameter for ExplainableBoostingMachine
ParamEBM = [
    {
        'EBM__max_bins': ParameterEbmMaxBins,
        'EBM__max_interaction_bins': ParameterEbmMaxInteractionBins,
        'EBM__max_rounds': ParameterEbmMaxRounds,
        'EBM__learning_rate': ParameterEbmLearningRate,
        'EBM__min_samples_leaf': ParameterEbmMinSampleLeaf,
        'EBM__outer_bags': ParameterEbmOuterBags,
        'EBM__validation_size': ParameterEbmValidationSize
    }    
]

 # set grip parameter for XGBoosting
ParamXGB = [
    {   
        'XGB__eta': ParameterXgbeta,
        'XGB__max_depth': ParameterXgbmax_depth,
        'XGB__min_child_weight': ParameterXgbmin_child_weight,
        'XGB__subsample': ParameterXgbsubsample,
        'XGB__colsample_bytree': ParameterXgbcolsample_bytree,
        'XGB__gamma': ParameterXgbgamma,
        'XGB__n_estimators': ParameterXgbn_estimators
    }
]

#####################################################################################################

# Pipelines for RandomForestClassifier
PipelineRF = Pipeline([
    ('scl', StandardScaler()),
    ('RF',RandomForestClassifier(random_state=rs))]
)

# Pipelines for LogisticRegression
PipelineLR = Pipeline([
    ('scl', StandardScaler()),
    ('LR', LogisticRegression(random_state=rs))]
)

# Pipelines for KNeighborsClassifier
PipelineKNN = Pipeline([
    ('scl', StandardScaler()),
    ('KNN', KNeighborsClassifier())]
)

# Pipelines for SVM
PipelineSVM = Pipeline([
    ('scl', StandardScaler()),
    ('SVM', svm.SVC(random_state=rs))]
)

# Pipelines for GradientBoostingClassifier
PipelineGBC = Pipeline([
    ('scl', StandardScaler()),
    ('GBC', GradientBoostingClassifier(random_state=rs))]
)
    
# Pipelines for ExplainableBoostingClassifier
PipelineEBM = Pipeline([
    ('scl', StandardScaler()),
    ('EBM', ExplainableBoostingClassifier(random_state=rs, interactions=0))]
)

# Pipelines for RandomForestClassifier
PipelineXGB = Pipeline([
    ('scl', StandardScaler()),
    ('XGB',xgb.XGBClassifier(objective='multi:softprob', use_label_encoder=False, eval_metric='mlogloss', num_class=3))]
)

#####################################################################################################

# Search for Random Forest Classifier
SearchRF = HalvingGridSearchCV(estimator=PipelineRF, param_grid=ParamRFC, scoring=scoring, cv=cv, verbose=vb, n_jobs=jobs)

# Seach for Logistic Regression
SearchLR = HalvingGridSearchCV(estimator=PipelineLR, param_grid=ParamLR, scoring=scoring,cv=cv, verbose=vb)

# Search for K-Neighbors Classifier 
SearchKNN = HalvingGridSearchCV(estimator=PipelineKNN, param_grid=ParamKNN, scoring=scoring, cv=cv, verbose=vb)

# Search for Support Vektor Machine
SearchSVM = HalvingGridSearchCV(estimator=PipelineSVM, param_grid=ParamSVC, scoring=scoring, cv=cv, verbose=vb, n_jobs=jobs)

# Search for Gradient Boosting Classifier
SearchGBC = HalvingGridSearchCV(estimator=PipelineGBC, param_grid=ParamGBC, scoring=scoring, cv=cv, verbose=vb, n_jobs=jobs)

# Search for Explainable Boosting Machine
# SearchEBM = ExplainableBoostingClassifier(estimator=PipelineEBM, param_grid=ParamEBM, scoring=scoring, cv=cv, verbose=vb, n_jobs=jobs)
SearchEBM = HalvingGridSearchCV(estimator=PipelineEBM, param_grid=ParamEBM, scoring=scoring, cv=cv, verbose=vb, n_jobs=jobs)

# Search for XGBoosting
SearchXGB = HalvingGridSearchCV(estimator=PipelineXGB, param_grid=ParamXGB, scoring=scoring, cv=cv, verbose=vb)
    
#####################################################################################################

# List with estimatiors
SearchGrid = [SearchRF, SearchLR, SearchKNN, SearchSVM, SearchGBC, SearchEBM, SearchXGB]


# HalvingGridSearch

In [ ]:
# open scenario mapping file to read
dfScenario = pd.read_csv(PathConfig + FileScenario, index_col=0)

# loop through scenario dataframe
for ind in dfScenario.index:

    # get scenarios and dataset from dataframe
    Scenario = (dfScenario['Scenario'][ind])
    
    # load scenario file
    df = pd.read_csv(PathScenario + Scenario + '_ah.csv')

    # Select data and remove target
    X = df.drop("label",axis=1)

    # Select target
    y = df['label']

    # Split dataset to train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

    # Print line
    print(80 * '=')

    # Print scenario name
    print (Scenario)

    # Print line
    print(80 * '-')

    # Set counter to 0
    counter = 0

    # loop through SearchGrid
    for pipe in SearchGrid:

        # Fit model
        pipe.fit(X_train,y_train)

        # Print results
        print('{} Test Accuracy: {}'.format(EstimatorNames[counter], pipe.score(X_test,y_test)))
        print('{} Best Params: {}'.format(EstimatorNames[counter], pipe.best_params_))
        print(80 * '-')    
        
        # Increase counter for estimatior name
        counter +=1
    
    # Print line
    print(60 * '-')

    # Increase estimatior counter
    EstimatorNamesCounter +=1 

# HalvingGridSearch for a single file and estimator

In [ ]:
# Settings for this single script
# Beginn

# First: set scenario file
Scenario = 'Scenario01'

# Second: define estimator
SearchGrid = [SearchXGB]

# Third: set True if estimator is XGB
CheckEstimatorXgb = True

# Counter for Estimator name
# 0: 'RandomForestClassifier', 
# 1: 'LogisticRegression', 
# 2: 'KNeighborsClassifier', 
# 3: 'SVC', 
# 4: 'GradientBoostingClassifier',
# 5: 'ExplainableBoostingClassifier',
# 6: 'XGBClassifier
counter = 6

# End
# ----------------------------------------------------------------------------------------

# load scenario file
df = pd.read_csv(PathScenario + Scenario + '_ah.csv')

# Select data and remove target
X = df.drop("label", axis = 1)

# XGB need other labels
# xgboost needs labels from 0 to ... n
if CheckEstimatorXgb:

    # First: get names of unique classes and save it into classes
    classes = df['label'].unique()

    # Second: create dictionary with original y values
    classesDictionary = {i: value for i, value in enumerate(classes)}

    # Third: replace y-value with values beginning from 0 to ...
    for key, value in classesDictionary.items():
        df.loc[df['label'] == value, 'label'] = key

# Select target
y = df['label']

# Split dataset to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

# Print line
print(80 * '=')

# Print Scenario name
print (Scenario)

# Print line
print(80 * '-')

# Fit model
pipe = SearchGrid[0]
pipe.fit(X_train,y_train)

# Print results
print('{} Test Accuracy: {}'.format(EstimatorNames[counter], pipe.score(X_test,y_test)))
print('{} Best Params: {}'.format(EstimatorNames[counter], pipe.best_params_))
print(80 * '-')    

# Print line
print(60 * '-')
